# QUIZ 2 - Ciencia de Datos Aplicada
Juan Felipe Palacios - 201616389

In [58]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [59]:
df_insurance = pd.read_csv('https://raw.githubusercontent.com/jufepalacios/Ciencia_Datos_Aplicada/main/Quiz_2/insurance.csv')

In [60]:
df_insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [61]:
df_insurance.shape

(1338, 7)

In [62]:
df_insurance = df_insurance.drop_duplicates()
df_insurance.shape

(1337, 7)

In [63]:
df_insurance['sex'].unique()

array(['female', 'male'], dtype=object)

In [64]:
df_insurance['smoker'].unique()

array(['yes', 'no'], dtype=object)

In [65]:
df_insurance['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [66]:
def change_sex(row):
    if row == 'female':
        return 0
    else:
        return 1

def change_smoker(row):
    if row == 'no':
        return 0
    else:
        return 1

In [67]:
df_insurance['sex'] = df_insurance['sex'].apply(change_sex)
df_insurance['smoker'] = df_insurance['smoker'].apply(change_smoker)
df_insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [68]:
df_regiones = pd.get_dummies(df_insurance['region'])
df_insurance = pd.concat([df_insurance,df_regiones], axis = 1).drop('region',axis=1)

In [69]:
df_insurance.head()

,age,sex,bmi,children,smoker,charges,northeast,northwest,southeast,southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


In [70]:
X = df_insurance.drop('charges',axis=1)
Y = df_insurance['charges']

In [71]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=1)

In [72]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [73]:
lin_reg = LinearRegression()
model = lin_reg.fit(X_train_scaled,Y_train)

print('intercept: {}'.format(model.intercept_))

print('coefficients: {}'.format(model.coef_))

intercept: 13628.60348397371
coefficients: [ 3.57244870e+03 -2.47637011e+02  2.32906515e+03  6.84278536e+02
  9.78803550e+03  4.12644505e+16  4.05679269e+16  4.23910035e+16
  4.10018459e+16]


In [74]:
preds_train = model.predict(X_train_scaled)
preds_test = model.predict(X_test_scaled)

In [75]:
print(mean_absolute_error(Y_train, preds_train), mean_absolute_error(Y_test, preds_test))
print(np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test)))


4362.145500357375 3924.0010120606894
6145.395806774114 5688.771688881152


In [76]:
df_insurance['charges'].describe()

count     1337.000000
mean     13279.121487
std      12110.359656
min       1121.873900
25%       4746.344000
50%       9386.161300
75%      16657.717450
max      63770.428010
Name: charges, dtype: float64

Entrene un primer modelo de regresión lineal sin aplicar ningún tipo de regularización. Evalúe dicho modelo y
concluya:

1.   ¿Es aceptable el error obtenido?
2.   ¿Hay evidencia de overfitting?

1. El error MAE y RSEM para el set de datos de prueba dan $3924$ y $5687$ dólares respectivamente. De esto se considera que los errores obtenidos son bajos y es aceptable el modelo obtenido.

2. Como los errores obtenidos para los set de datos de test dan menores que los de entrenamiento, se concluye en este caso que no hay overfitting.

Aplique una transformación polinomial a los datos de entrada y regularización Ridge o Lasso al modelo de
regresión. Pruebe con al menos 2 grados diferentes del polinomio y con al menos 3 valores de alpha para la
regularización. Evalúe dichos modelos y concluya:


In [77]:
poly_features_2 = PolynomialFeatures(degree=2, include_bias=False)
poly_features_3 = PolynomialFeatures(degree=3, include_bias=False)

X_poly_2_train = poly_features_2.fit_transform(X_train_scaled)
X_poly_3_train = poly_features_3.fit_transform(X_train_scaled)

X_poly_2_test = poly_features_2.fit_transform(X_test_scaled)
X_poly_3_test = poly_features_3.fit_transform(X_test_scaled)

In [78]:
alphas = [0.1,0.3,0.5,0.7,0.9]

# Poly 2
for alpha in alphas:
    print('Ridge para un alpha = {}'.format(alpha))
    ridge_reg = Ridge(alpha=alpha, solver='cholesky')
    ridge_reg.fit(X_poly_2_train,Y_train)

    preds_train = ridge_reg.predict(X_poly_2_train)
    preds_test = ridge_reg.predict(X_poly_2_test)

    print('MAE train = {}'.format(mean_absolute_error(Y_train, preds_train)))
    print('MAE test = {}'.format(mean_absolute_error(Y_test, preds_test)))
    print('RMSE train = {}'.format(np.sqrt(mean_squared_error(Y_train, preds_train))))
    print('RMSE test = {}'.format(np.sqrt(mean_squared_error(Y_test, preds_test))))
    print('\n')

Ridge para un alpha = 0.1
MAE train = 2977.3932244331927
MAE test = 2667.796928034085
RMSE train = 4769.56608035549
RMSE test = 4666.116779306431


Ridge para un alpha = 0.3
MAE train = 2977.5238324126144
MAE test = 2667.919640965315
RMSE train = 4769.56646593429
RMSE test = 4665.985521276239


Ridge para un alpha = 0.5
MAE train = 2977.65684172988
MAE test = 2668.04241599721
RMSE train = 4769.567236656331
RMSE test = 4665.854738997181


Ridge para un alpha = 0.7
MAE train = 2977.7942495224997
MAE test = 2668.16525302692
RMSE train = 4769.56839206766
RMSE test = 4665.7244319253505


Ridge para un alpha = 0.9
MAE train = 2977.9316551286756
MAE test = 2668.2917010788997
RMSE train = 4769.569931714896
RMSE test = 4665.594599517543




In [79]:
# Poly 3
for alpha in alphas:
    print('Ridge para un alpha = {}'.format(alpha))
    ridge_reg = Ridge(alpha=alpha, solver='cholesky')
    ridge_reg.fit(X_poly_3_train,Y_train)

    preds_train = ridge_reg.predict(X_poly_3_train)
    preds_test = ridge_reg.predict(X_poly_3_test)

    print('MAE train = {}'.format(mean_absolute_error(Y_train, preds_train)))
    print('MAE test = {}'.format(mean_absolute_error(Y_test, preds_test)))
    print('RMSE train = {}'.format(np.sqrt(mean_squared_error(Y_train, preds_train))))
    print('RMSE test = {}'.format(np.sqrt(mean_squared_error(Y_test, preds_test))))
    print('\n')

Ridge para un alpha = 0.1
MAE train = 2865.8698969445595
MAE test = 2999.9348123878635
RMSE train = 4560.335138068824
RMSE test = 4932.484059650113


Ridge para un alpha = 0.3
MAE train = 2865.8778119295375
MAE test = 2999.758136906438
RMSE train = 4560.335278015869
RMSE test = 4932.2499576906


Ridge para un alpha = 0.5
MAE train = 2865.8857452576917
MAE test = 2999.581588843138
RMSE train = 4560.335557532631
RMSE test = 4932.0162432564575


Ridge para un alpha = 0.7
MAE train = 2865.893696877637
MAE test = 2999.405168005435
RMSE train = 4560.335976226766
RMSE test = 4931.782915437062


Ridge para un alpha = 0.9
MAE train = 2865.901666738219
MAE test = 2999.2288742013343
RMSE train = 4560.336533707375
RMSE test = 4931.549973324832




1. ¿Fue posible mejorar el error? ¿Qué hiper-parámetros tiene el modelo que produce el menor error?
2. ¿Qué atributos parecen ser los más importantes para realizar la predicción?

1. Despues de la transformación y la regularización si se obtuvieron menores errores. Tambien se peude observar que a medida que aumenta el alpha en la regularizacion, loes errores disminuyen para las misma transformacion polinomial.

  Por otra parte, para un mismo valor de alpha, se obtienen menores errores con una transformacion polinomial de grado 2 que de grado 3.

  Finalmente, los hiper-paramerntros del modelo con menos error son: un polinomio de grado 2 y un alpha de 0.9 para la regularizacion.


2. Los atributos que parecen ser los más importantes para realizar la prediccion son: la region, si fuma o no y la edad (sucesivamente de mayor a menor)